# OpenCV Edge Detection Script

## Set up and imports

In [ ]:
!pip install opencv-python

In [ ]:
!git clone https://github.com/acarcher/hed-opencv-dl.git

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import os, shutil
import numpy as np
from PIL import Image

## Crop Layer

In [ ]:
class CropLayer(object):
    def __init__(self, params, blobs):
        # initialize our starting and ending (x, y)-coordinates of the crop
        self.startX = 0
        self.startY = 0
        self.endX = 0
        self.endY = 0

    def getMemoryShapes(self, inputs):
        # the crop layer will receive two inputs -- we need to crop
        # the first input blob to match the shape of the second one,
        # keeping the batch size and number of channels
        (inputShape, targetShape) = (inputs[0], inputs[1])
        (batchSize, numChannels) = (inputShape[0], inputShape[1])
        (H, W) = (targetShape[2], targetShape[3])

        # compute the starting and ending crop coordinates
        self.startX = int((inputShape[3] - targetShape[3]) / 2)
        self.startY = int((inputShape[2] - targetShape[2]) / 2)
        self.endX = self.startX + W
        self.endY = self.startY + H

        # return the shape of the volume (we'll perform the actual
        # crop during the forward pass)
        return [[batchSize, numChannels, H, W]]

    def forward(self, inputs):
        # use the derviced (x, y)-coordinates to perform the crop
        return [inputs[0][:, :, self.startY:self.endY,
                                self.startX:self.endX]]

## Datasets 
In case there are any other dataset in your directory, you can add it to the list, and the script will add the ".npy" output to the folder.

We recommend copying your datasets into a new folder and then run this script as it will add the ".npy" files into the same directory.

In [ ]:
datasets = ['Kvasir Seg', 'ETIS-LaribPolypDB', 'CVC-ColonDB']

# Generating edges and creating the 4th layer

In [ ]:
# loading weights and the model
net = cv2.dnn.readNetFromCaffe("/content/hed-opencv-dl/hed_model/deploy.prototxt", "/content/hed-opencv-dl/hed_model/hed_pretrained_bsds.caffemodel")
cv2.dnn_registerLayer("Crop", CropLayer)

for dataset in datasets:
  train_path = '/content/drive/MyDrive/CSC490 Datasets/FINAL_COMBINED_DATASET_EDGES/Training dataset/{}/images/'.format(dataset)
  test_path = '/content/drive/MyDrive/CSC490 Datasets/FINAL_COMBINED_DATASET_EDGES/Test dataset/{}/images/'.format(dataset)

  train_images = list(os.listdir(train_path))
  test_images = list(os.listdir(test_path))

  # setting up train images
  for train_image_name in train_images:
    if (".npy" in train_image_name or train_image_name[:-3] + 'npy' in train_images):
      continue
    src_path = os.path.join(train_path, train_image_name)
    img = cv2.imread(src_path)
    (H, W) = img.shape[:2]

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    canny = cv2.Canny(blurred, 30, 150)

    blob = cv2.dnn.blobFromImage(img, scalefactor=1.0, size=(W, H),
                                mean=(104.00698794, 116.66876762, 122.67891434),
                                swapRB=False, crop=False)

    net.setInput(blob)
    hed = net.forward()
    hed = cv2.resize(hed[0, 0], (W, H))
    hed = (255 * hed).astype("uint8")

    hed = hed.reshape(hed.shape[0], hed.shape[1], 1)
    fourth_layer = np.concatenate((img, hed), axis=2)

    np.save(src_path[:-3] + 'npy', fourth_layer)

    print(src_path)
    

  # setting up for test images
  for test_image_name in test_images:
    src_path = os.path.join(test_path, test_image_name)
    img = cv2.imread(src_path)
    (H, W) = img.shape[:2]

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    canny = cv2.Canny(blurred, 30, 150)

    blob = cv2.dnn.blobFromImage(img, scalefactor=1.0, size=(W, H),
                                mean=(104.00698794, 116.66876762, 122.67891434),
                                swapRB=False, crop=False)

    net.setInput(blob)
    hed = net.forward()
    hed = cv2.resize(hed[0, 0], (W, H))
    hed = (255 * hed).astype("uint8")

    hed = hed.reshape(hed.shape[0], hed.shape[1], 1)
    fourth_layer = np.concatenate((img, hed), axis=2)

    np.save(src_path[:-3] + 'npy', fourth_layer)


# Removing the ".jpg" files

If you want to just have the ".npy" files in the folder, you can run the script below to achieve this result. 

In [ ]:
for dataset in datasets:
  train_path = '/content/drive/MyDrive/CSC490 Datasets/FINAL_COMBINED_DATASET_EDGES/Training dataset/{}/images/'.format(dataset)
  test_path = '/content/drive/MyDrive/CSC490 Datasets/FINAL_COMBINED_DATASET_EDGES/Test dataset/{}/images/'.format(dataset)

  train_images = list(os.listdir(train_path))
  test_images = list(os.listdir(test_path))

  # deleting jpgs from train data
  for train_image_name in train_images:
    if (train_image_name[-3:] == "jpg"):
      src_path = os.path.join(train_path, train_image_name)
      os.remove(src_path)

  # deleting jpgs from test data
  for test_image_name in test_images:
    if (test_image_name[-3:] == "jpg"):
      src_path = os.path.join(test_path, test_image_name)
      os.remove(src_path)

  